# Exploring multiple approaches to machine learning across multiple small EHD experimental datasets
 - The task is to predict (regress) the size of printed features, as a function of waveform inputs to the EHD printer
 - A second task is to predict (classify) whether a given waveform input will produce any printed pattern at all. (This is equivalent to understanding the print onset voltage threshold.)
 - Multiple hidden confounding variables are likely, such as ink/tip/substrate/atmospheric condition; ink and tip clogging; print tip height from the substrate. Some of these will be relatively constant for each run of experiments, some will vary from feature to feature.
 - Ink dynamics at the printing tip are complex and nonlinear, with electrical and fluid/acoustic phenomena that interact with each other.

## Goals
 - In regression, aim for <3% mean absolute error (from the predicted outcome)
 - In classification, aim for >0.9 ROC AUC
 - For a new/test set, achieve these in <=100 experiments

In [1]:
import sys
import pickle

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from scipy.stats import pearsonr

sys.path.append('..')
from ehd_dataset import EHD_Loader
from ehd_models import EHD_Model


%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

NEW_CACHE = True
# TODO option to bypass the cache op for nondestructive testing

# Print Area Regression Models <<<<<<<<<<<<<<<<<<<<
REGRESSION_model_architectures = {'MLE': {},
                                  # 'cold_MLP': {'max_iter': 100_000},
                                  # 'lastXY_MLP': {'max_iter': 200_000},
                                  'all_pretrained_RF': {},
                                  # 'lastXY_RF': {},
                                  'cold_RF': {}}

# Jetting Classification Models <<<<<<<<<<<<<<<<<<<<
CLASS_model_architectures = {'MLE_class': {},
                             # 'cold_MLP_class': {'max_iter': 100_000},
                             # 'lastXY_MLP_class': {'max_iter': 200_000},
                             'all_pretrained_RF_class': {},
                             # 'lastXY_RF_class': {},
                             'cold_RF_class': {}}

XTYPE = "square"
YTYPE = "diameter"

In [ ]:
INDEX = "C:/Dropbox/SPEED/Self Driving EHD/Datasets/dataset_index.xlsx"
# INDEX = "C:/Users/Oliver/Dropbox/SPEED/Self Driving EHD/Datasets/dataset_index.xlsx"
loader = EHD_Loader(INDEX)
print("Datasets Loaded!\n>> Quick correlation validation check -- [auac; vec L2] <<")

for i, df in enumerate(loader.datasets):
    AUAC, _ = pearsonr(df.area,
                       df.wave.apply(lambda x: np.sum(np.abs(x))))
    VL2, _ = pearsonr(df.area,
                       df.vector.apply(lambda x: np.sqrt(np.sum(x**2))))
    print(f"<<{loader.names[i]} -- [{AUAC:.3f}; {VL2:.3f}]", end='>> ')

Failed to load 10-Mar-2022 large nozzle mosaic: 'DataFrame' object has no attribute 'note'
dataset C:\Dropbox\SPEED\Self Driving EHD\Datasets\29-Mar-2022 lg 1cm 300 points	263 points	offset 2	corr 0.4979414348873561
> c:\dropbox\python\ehd_exsitu\ehd_dataset\dataset.py(111)__init__()
    110 
--> 111                 self.datasets.append(df)
    112                 self.names.append(os.path.basename(row['Path']))



ipdb>  df.head()


   obj_count           area  print_length   max_width  mean_width  \
0          2   17899.985347    201.025641   28.717949   21.161695   
1          0       0.000000      0.000000    0.000000    0.000000   
2          1  138870.954565    281.823136  153.094451  117.107313   
3          1   27257.198260    225.710919   38.605398   28.699735   
4          6     371.809785    127.179487    6.153846    0.694789   

                                                wave  \
0  [0.0, -16.7835354, -33.49617624, -50.067351599...   
1  [0.0, 7.9345913999999995, 15.83229828, 23.6564...   
2  [0.0, -15.240176279999998, -30.41114928, -45.4...   
3  [0.0, 14.27829036, 28.480577519999997, 42.5312...   
4  [0.0, 7.17114204, 14.34002904, 21.50435028, 28...   

                                              vector  volts  jetted SIJ Tip  \
0  [-211.46407223999998, 234.89078844000002, -167...  348.0    True   Large   
1  [125.14207019999999, 86.15063291999999, -87.69...  348.0   False   Large   
2  [-165.51

ipdb>  row


Path                                     29-Mar-2022 lg 1cm 300 points
Date                                               2022-03-29 00:00:00
SIJ Tip                                                          Large
Standoff [um]                                                     20.0
Velocity [um/s]                                                  100.0
Wavegen                                                      harmonics
Samples                                                          263.0
Hz                                                                 4.0
V Thresh [V] @ .5s                                                 NaN
W thresh [s] @ 1.5 Vt                                              NaN
V at 0.7 scale (go a little lower)                                 NaN
width @ 4 Hz (little lower to higher)                              NaN
Starting V                                                         NaN
Final V                                                            NaN
Name: 

ipdb>  q


Failed to load 29-Mar-2022 lg 1cm 300 points: 
dataset C:\Dropbox\SPEED\Self Driving EHD\Datasets\2-May-2022__run 1	121 points	offset 32	corr 0.6417618745477631
> c:\dropbox\python\ehd_exsitu\ehd_dataset\dataset.py(111)__init__()
    110 
--> 111                 self.datasets.append(df)
    112                 self.names.append(os.path.basename(row['Path']))



In [5]:
model = EHD_Model(architecture='all_pretrained_RF', params={})
print(loader.datasets[0].head())

   obj_count           area  print_length   max_width  mean_width  \
0          2   17899.985347    201.025641   28.717949   21.161695   
1          0       0.000000      0.000000    0.000000    0.000000   
2          1  138870.954565    281.823136  153.094451  117.107313   
3          1   27257.198260    225.710919   38.605398   28.699735   
4          6     371.809785    127.179487    6.153846    0.694789   

                                                wave  \
0  [0.0, -16.7835354, -33.49617624, -50.067351599...   
1  [0.0, 7.9345913999999995, 15.83229828, 23.6564...   
2  [0.0, -15.240176279999998, -30.41114928, -45.4...   
3  [0.0, 14.27829036, 28.480577519999997, 42.5312...   
4  [0.0, 7.17114204, 14.34002904, 21.50435028, 28...   

                                              vector  volts  jetted  
0  [-211.46407223999998, 234.89078844000002, -167...  348.0    True  
1  [125.14207019999999, 86.15063291999999, -87.69...  348.0   False  
2  [-165.51479952, -185.0214906, -124.

In [3]:
from ehd_models import EHD_Model

CACHE_NAME = 'regression.cache'

if NEW_CACHE:
    reg_results = []
    reg_names = []
else:
    with open(CACHE_NAME, 'rb') as f:
        (reg_results, reg_names) = pickle.load(f)

for architecture, params in REGRESSION_model_architectures.items():
    if architecture in reg_names:
        print(f'{architecture} was already evaluated - loaded from cache')
    else:
        print(f'\nEvaluating regression model type: {architecture}')
        model = EHD_Model(architecture=architecture, params=params)

        # for dataset_fold in range(loader.num_folds()):
        for dataset_fold in range(loader.num_folds(model.filters)):
            print(f"fold {dataset_fold}", end=" ")
            pretrain_set, eval_set, eval_name = loader.folded_dataset(fold=dataset_fold, xtype=model.xtype, ytype=model.ytype,
                                                                     pretrain=model.pretrainer, filters=model.filters)
            # try:
            if model.pretrainer:
                print('pretrain...', end=' ')
                model.pretrain(pretrain_set)
                print('done', end=' ')

            output = model.evaluate(eval_set)

            output['architecture'] = architecture
            output['eval_dataset'] = eval_name
            reg_names.append(architecture)
            reg_results.append(output)

with open(CACHE_NAME, 'wb') as f:
    pickle.dump((reg_results, reg_names), f)

FileNotFoundError: [Errno 2] No such file or directory: 'regression.cache'

In [ ]:
from ehd_models import EHD_Model

CACHE_NAME = 'classification.cache'

if NEW_CACHE:
    class_results = []
    class_names = []
else:
    with open(CACHE_NAME, 'rb') as f:
        (class_results, class_names) = pickle.load(f)
    
for architecture, params in CLASS_model_architectures.items():
    if architecture in class_names:
        print(f'{architecture} was already evaluated - loaded from cache')
    else:
        print(f'\nEvaluating classification model type: {architecture}')
        model = EHD_Model(architecture=architecture, params=params)

        for dataset_fold in range(loader.num_folds(model.filters)):
            print(f"fold {dataset_fold}", end=" ")
            pretrain_set, eval_set, eval_name = loader.folded_dataset(fold=dataset_fold, xtype=model.xtype, ytype=model.ytype,
                                                                     pretrain=model.pretrainer, filters=model.filters)
            model.pretrain(pretrain_set)
            output = model.evaluate(eval_set)

            output['architecture'] = architecture
            output['eval_dataset'] = eval_name
            class_names.append(architecture)
            class_results.append(output)

with open(CACHE_NAME, 'wb') as f:
    pickle.dump((class_results, class_names), f)

In [ ]:
import seaborn as sns

OUT_XLSX = 'eval_summary.xlsx'
xlsx_writer = pd.ExcelWriter(OUT_XLSX)
plt.rcParams['figure.figsize'] = (11.0, 10.0) # set default size of plots
plt.clf()

def n_walk_result_vis(df, x, y, trends, ax=None, outfile=None, legend='auto', labels=None):
    bar_ci = 0.95  # 'sd' for standard deviation
    sns.lineplot(
        data=df.loc[~df['unique N']],
        x=x, y=y, hue=trends, style=trends,
        markers=True,
        err_style="bars", ci=bar_ci,
        markersize=7,
        linewidth=1.8,
        ax=ax,
        legend=legend
    )
    ticks = df[x].loc[~df['unique N']].unique()
    ax.set_xticks(ticks=ticks,
               labels=np.round(10**ticks).astype(int))
    ax.set_xlabel('dataset size')
    if legend is not None and labels is not None:
        ax.legend(labels)
    if y == 'MSE':
        ax.set_ylim(0, 6e9)
    if y == 'MAE':
        ax.set_ylim(0, 60_000)
    if y == 'r':
        ax.set_ylim(0, 1)
    if y == 'F1':
        ax.set_ylim(0.49, 1)
    if y == 'AUC':
        ax.set_ylim(0.49, 1)
    if y == 'MAPE':
        ax.set_ylim(0, 100)
    if outfile is not None:
        plt.savefig(outfile, bbox_inches='tight', pad_inches=0.1, transparent=False, dpi=500)
        plt.clf()

# Expand, log, and vis regression results
df = pd.concat(reg_results, ignore_index=True)
df['log_size'] = df['train_size'].apply(lambda x: np.log10(x).round(3))
df['MAPE'] = df['MAPE'].apply(lambda x: 100 * x)
df['unique N'] = df['train_size'].apply(lambda x: (df['train_size'] == x).sum() <= len(df['architecture'].unique()))

fig, ax = plt.subplots(2,2)

legend = 'auto'
labels = ('MLE', 'MLP: last X/Y', 'Rand. Forest: pretrain only', 'Rand. Forest: last X/Y', 'Rand. Forest: cold start')
for i, metric in enumerate(['MAE', 'r']):  # MSLE and MAPE left out for now + 'MSE', 
    #n_walk_result_vis(df, x='log_size', y=metric, trends='architecture', outfile=f"{metric}.png")
    n_walk_result_vis(df, x='log_size', y=metric, trends='architecture', ax=ax[0][i], legend=legend, labels=labels)
    labels = None
    legend = False
    
df.to_excel(xlsx_writer, index=False, sheet_name='Regression')

# Expand, log, and vis classification results
df = pd.concat(class_results, ignore_index=True)
df['log_size'] = df['train_size'].apply(lambda x: np.log10(x).round(3))
df['unique N'] = df['train_size'].apply(lambda x: (df['train_size'] == x).sum() <= len(df['architecture'].unique()))
    
df.to_excel(xlsx_writer, index=False, sheet_name='Classification')

legend = 'auto'
labels = ('MLE', 'MLP: last X/Y', 'Rand. Forest: pretrain only', 'Rand. Forest: last X/Y', 'Rand. Forest: cold start', 'MLP: cold start')
for i, metric in enumerate(['AUC', 'F1']):
    #n_walk_result_vis(df, x='log_size', y=metric, trends='architecture', outfile=f"{metric}.png")
    n_walk_result_vis(df, x='log_size', y=metric, trends='architecture', ax=ax[1][i], legend=legend, labels=labels)
    labels = None
    legend = False


plt.savefig('4-square_metrics.png', bbox_inches='tight', pad_inches=0.1, transparent=False, dpi=300)
plt.clf()
xlsx_writer.save()
xlsx_writer.close()